In [ ]:
lab10-2

# lab 10-3 : dropout


In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
import datetime

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [20]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """ #초모수값 지정
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size #468
label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).repeat()
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

train_dataset=train_dataset.shuffle(100000)
test_dataset =test_dataset.shuffle(100000)
##참고:https://stackoverflow.com/questions/50437234/tensorflow-dataset-shuffle-then-batch-or-batch-then-shuffle
#train_iterator = train_dataset.make_one_shot_iterator() 대신 batch에 대해 shuffle!
#test_iterator = test_dataset.make_one_shot_iterator()

In [7]:
###수정!!



class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        ###Xavier initialization 이용
        weight_init = tf.kerasv.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(512,kernel_initializer=weight_init, input_shape=(784,),activation="relu"))
        self.model.add(tf.keras.layers.Dropout(rate=0.5))

        for i in range(3) :
               self.model.add(tf.keras.layers.Dense(512, kernel_initializer=weight_init))
               self.model.add(relu())
               self.model.add(tf.keras.layers.Dropout(rate=0.5))


        self.model.add(tf.keras.layers.Dense(label_dim, kernel_initializer=weight_init,activation='softmax'))
        self.model.compile(optimizer='adam',
                loss=custom_loss,
                metrics=['accuracy'])
    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x
    
# Create a loss function
def custom_loss(y_true, y_pred):
    return  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_pred,y_true))


In [13]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512,kernel_initializer=weight_init, input_shape=(784,),activation="relu"))
    model.add(tf.keras.layers.Dropout(rate=0.5))
    
    for i in range(3) :
        model.add(tf.keras.layers.Dense(512, kernel_initializer=weight_init))
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.Dropout(rate=0.5))

    model.add(tf.keras.layers.Dense(label_dim,kernel_initializer= weight_init,activation='softmax'))
    
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    return model

model = create_model_function(label_dim)
#model.summary()

In [ ]:
#for checkpoint
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_10_3/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq=1)

model.fit(train_dataset, epochs =1,
          validation_data =test_dataset,steps_per_epoch=training_iterations,
          callbacks = [cp_callback],validation_steps=1)  # 훈련 단계에 콜백을 전달합니다


============================================================================================================

In [17]:
tf.random.shuffle(
range(10)
)
train_dataset[1:2,]

TypeError: 'RepeatDataset' object is not subscriptable